In [ ]:
import joblib
import pandas as pd
import numpy as np

In [13]:

model = joblib.load("clf_model_ensemble_voter.pkl")
feature_columns = joblib.load("feature_columns.pkl")
te = joblib.load("target_encoder.pkl")
global_mean = joblib.load("target_global_mean.pkl")
train_medians = joblib.load("train_medians.pkl")

In [14]:
df=pd.read_csv("../Processed Data/prd_inference.csv")
df["main_category"] = te.transform(df["main_category"])
df["main_category"] = df["main_category"].fillna(global_mean)
df["has_review_text"] = df["has_review_text"].fillna(0)

cols_to_impute = [
    'n_distinct_categories',
    'avg_photos_per_product',
    'avg_desc_length',
    'avg_weight',
    'avg_volume',
    'review_count',
    'avg_review_score',
    'avg_review_length',
    'freight_ordervalue_ratio',
    'pymt_mode_boleto',
    'pymt_mode_credit_card',
    'pymt_mode_debit_card',
    'pymt_mode_not_defined',
    'pymt_mode_voucher',
    'total_payment_installments',
    'price_spread_ratio',
    'weight_spread_ratio',
    'volume_spread_ratio'
]
df[cols_to_impute] = df[cols_to_impute].fillna(train_medians)

In [15]:
X_inference = df[feature_columns]
customer_ids = df["customer_unique_id"]

In [16]:
probs = model.predict_proba(X_inference)[:, 1]

In [17]:
results = df[["customer_id"]].copy()
results["repeat_probability"] = probs


In [18]:
results

,customer_id,repeat_probability
0,08c5351a6aca1c1589a38f244edeee9d,0.500901
1,86dc2ffce2dfff336de2f386a786e574,0.404542
2,355077684019f7f60a031656bd7262b8,0.532391
3,7ec40b22510fdbea1b08921dd39e63d8,0.518737
4,6f989332712d3222b6571b1cf5b835ce,0.472553
...,...,...
94985,e450a297a7bc6839ceb0cf1a2377fa02,0.376568
94986,e60df9449653a95af4549bbfcb18a6eb,0.389437
94987,6e353700bc7bcdf6ebc15d6de16d7002,0.478059
94988,496630b6740bcca28fce9ba50d8a26ef,0.395953


In [19]:
results.to_csv("../Processed Data/InferenceResults.csv", index=False)